# Урок 1: Quantization-Aware Training (QAT)

В этом уроке мы разберем «золотой стандарт» квантования — обучение с учетом ограничений точности. В отличие от Post-Training Quantization (PTQ), где мы сжимаем уже готовую модель, QAT позволяет нейросети «адаптироваться» к шуму квантования в процессе обучения.

## 1. Математическая теория

### 1.1. Проблема дифференцируемости
Операция квантования (округления) является ступенчатой функцией:
$$q(x) = \text{round}(x / s) \cdot s$$
Производная такой функции везде равна нулю (или не определена в точках скачка). Это делает стандартный метод обратного распространения ошибки (Backpropagation) невозможным.

### 1.2. Straight-Through Estimator (STE)
Для решения проблемы используется «трюк» STE: во время прямого прохода (forward) мы используем квантованные веса, а во время обратного прохода (backward) притворяемся, что квантования не было, и пропускаем градиент без изменений:
$$\frac{\partial \mathcal{L}}{\partial x} \approx \frac{\partial \mathcal{L}}{\partial q(x)}$$

### 1.3. Методы из обзора:
*   **LLM-QAT (Liu et al., 2023b):** Предлагает использовать дистилляцию данных (Data-free) — модель-учитель генерирует тексты, на которых обучается квантованная модель-ученик. Это решает проблему отсутствия чистых данных для дообучения.
*   **BitDistiller (Du et al., 2024):** Фокусируется на стабильности при экстремально низких битах (ниже 4-х). Использует асимметричное квантование и доверительную дистилляцию (Confidence-Aware KL-Divergence).
*   **OneBit (Xu et al., 2024):** Подход для 1-битного сжатия. Веса представляются как $W \approx S \cdot B$, где $B \in \{-1, 1\}^{M \times N}$ — бинарная матрица, а $S$ — обучаемый вектор масштаба высокого разрешения.

---

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from src.model import GPTLanguageModel, device

class STEQuantize(torch.autograd.Function):
    @staticmethod
    def forward(ctx, input, scale):
        q_x = torch.round(input / scale).clamp(-128, 127)
        return q_x * scale
    @staticmethod
    def backward(ctx, grad_output):
        return grad_output, None

class QATLinear(nn.Linear):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.scale = nn.Parameter(torch.tensor(0.01))
    def forward(self, x):
        q_weight = STEQuantize.apply(self.weight, self.scale.abs() + 1e-6)
        return F.linear(x, q_weight, self.bias)

print("nanoGPT Track: Реализована 'сырая' логика QAT через Linear Algebra & STE.")

## 2. Промышленная реализация: QLoRA
В современной индустрии полноценный QAT для моделей 7B+ слишком дорог. Вместо него используют **QLoRA** — комбинацию 4-битного квантования (NF4) и низкоранговых адаптеров (LoRA). Адаптеры обучаются в FP16, фактически выполняя роль QAT-коррекции.

In [ ]:
from transformers import BitsAndBytesConfig, AutoModelForCausalLM
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
)

try:
    model_id = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"
    model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map="auto")
    model = prepare_model_for_kbit_training(model)
    
    config = LoraConfig(
        r=8, lora_alpha=32, target_modules=["q_proj", "v_proj"],
        lora_dropout=0.05, bias="none", task_type="CAUSAL_LM"
    )
    model = get_peft_model(model, config)
    model.print_trainable_parameters()
    print("Llama Track: Промышленный QLoRA пайплайн (PEFT) готов.")
except Exception as e:
    print(f"Ошибка (нужен GPU): {e}")